In [ ]:
# install kaggle environment and upload from your computer the kaggle.json file (the API you had created before)
! pip install -q kaggle
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c gan-getting-started